# RAG Benchmark Evaluation on Natural Questions

This notebook evaluates a RAG (Retrieval-Augmented Generation) system on the Natural Questions dataset.

## Setup and Initialization

### Imports and dependencies

Import all necessary libraries for dataset loading, embeddings, LLM, text processing, and evaluation metrics.

In [ ]:
import os
import time

import pandas as pd
import json
import evaluate

from datetime import datetime
from langchain_core.messages import SystemMessage, HumanMessage
from tqdm import tqdm
from datasets import load_dataset
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.document import Document
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint, ChatHuggingFace
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

import faiss

### Config File

Configure the dataset split, output folder, embedding model, chunk size, overlap, and LLM parameters.

In [ ]:
config_file = "configs/custom_benchmark_config.json"

with open(config_file, "r") as f:
    config = json.load(f)

### Load API Key

Load the Hugging Face API key from environment variables to authenticate model requests.

In [ ]:
load_dotenv()
HF_API_KEY = os.getenv("API_KEY4")

### Load dataset

Load the Natural Questions dataset subset for benchmarking.

In [ ]:
dataset = load_dataset("natural_questions", split=config["dataset_split"])

## Model Initialization

### Embeddings initialization

Create the embedding model for converting text chunks into vector representations.

In [ ]:
embedding_model = HuggingFaceEmbeddings(
    model_name=config["embedding_model_name"]
)

### Text splitter configuration

Define how the document is split into overlapping chunks for embedding and retrieval.

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=config["chunk_size"],
    chunk_overlap=config["chunk_overlap"]
)

### LLM setup

Configure the language model endpoint on Hugging Face Hub for text generation.

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id=config["llm_model"],
    huggingfacehub_api_token=HF_API_KEY,
    task="text-generation",
    temperature=config["temperature"],
    max_new_tokens=config["max_new_tokens"],
)
chat_model = ChatHuggingFace(llm=llm)

### Retrieval and querying

Functions to retrieve relevant documents locally and query the LLM with that context.

In [ ]:
def retrieve_local(query, vectorstore, k=config["top_k"]):
    docs_faiss = vectorstore.similarity_search(query, k=k)
    return [(i, d.page_content) for i, d in enumerate(docs_faiss)]

In [ ]:
def get_relevant_doc_ids(split_docs, golden_answer=None):
    relevant_ids = []
    for doc in split_docs:
        content = doc.page_content.lower()
        if golden_answer and golden_answer.strip() and golden_answer.strip().lower() in content:
            relevant_ids.append(doc.metadata["doc_id"])
    return relevant_ids

In [ ]:
def ask(query, context):
    messages = [
        SystemMessage(content=f"Just answer queries based on {context}."),
        HumanMessage(content=f"""
        Answer the query: {query} based uniquely on the context: {context}, don't make up anything, just say what the context contains. If the information is not in the context, you must say you don't know. You must answer only the specified question and nothing else.
        """)
    ]
    response = chat_model.invoke(messages)
    return response.content

### Data extraction and preprocessing

Functions to extract valid answers from dataset.

In [ ]:
def extract_answers(sample):
    tokens = sample["document"]["tokens"]
    short_answer = ""
    start = sample["annotations"]["short_answers"][0]["start_token"]
    end = sample["annotations"]["short_answers"][0]["end_token"]
    if len(start) > 0:
        short_answer = " ".join([
            t for t, html in zip(tokens["token"][int(start[0]):int(end[0])], tokens["is_html"][start[0]:end[0]]) if not html
        ])

    long_answer = ""
    if sample["annotations"]["long_answer"][0]["start_token"] != -1:
        start = sample["annotations"]["long_answer"][0]["start_token"]
        end = sample["annotations"]["long_answer"][0]["end_token"]
        long_answer = " ".join([
            t for t, html in zip(tokens["token"][start:end], tokens["is_html"][start:end]) if not html
        ])

    return long_answer or "", short_answer or ""

In [ ]:
def preprocess_text(sample):
    tokens = sample["document"]["tokens"]
    return " ".join([t for t, html in zip(tokens["token"], tokens["is_html"]) if not html])

## Benchmarking 

### Retrieve benchmark

By finding the longest match between the prediction and the golden context, we can evaluate how well the model retrieves relevant information.

In [ ]:
def find_longest_match(str1, str2):
    max_len = 0
    len2 = len(str2)

    for i in range(len2):
        for j in range(i + 1, len2 + 1):
            substr = str2[i:j]
            if substr in str1 and len(substr) > max_len:
                max_len = len(substr)

    return max_len / len2 if len2 > 0 else 0


## Evaluation benchmark

Load rouge and bleu metrics for evaluating the model's predictions against the golden answers. 

In [ ]:
metric_rouge = evaluate.load("rouge")
metric_bleu = evaluate.load("bleu")

In [ ]:
import numpy as np

def precision_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    return len(set(retrieved_k) & set(relevant)) / k

def recall_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    return len(set(retrieved_k) & set(relevant)) / len(relevant) if relevant else 0

def average_precision(retrieved, relevant, k):
    score = 0.0
    num_hits = 0.0
    for i, doc_id in enumerate(retrieved[:k]):
        if doc_id in relevant:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    return score / len(relevant) if relevant else 0

def dcg_at_k(retrieved, relevant, k):
    dcg = 0.0
    for i, doc_id in enumerate(retrieved[:k]):
        if doc_id in relevant:
            dcg += 1.0 / np.log2(i + 2)
    return dcg

def ndcg_at_k(retrieved, relevant, k):
    dcg = dcg_at_k(retrieved, relevant, k)
    idcg = sum(1.0 / np.log2(i + 2) for i in range(min(len(relevant), k)))
    return dcg / idcg if idcg > 0 else 0

### Single sample processing

Process one sample: build context, eventually query LLM, extract answer, and compute metrics.

In [ ]:
def process_sample(i, sample):
    k_values = config.get("k_values", [1, 3, 5, 10])
    golden_context, golden_answer = extract_answers(sample)
    if golden_answer == "" or golden_context == "":
        return None

    query = sample["question"]["text"]
    text = preprocess_text(sample)
    docs = [Document(page_content=text)]
    split_docs = splitter.split_documents(docs)
    for idx, doc in enumerate(split_docs):
        doc.metadata = {"doc_id": f"doc_{idx}"}
    vectorstore = FAISS.from_documents(split_docs, embedding_model)

    docs_faiss = vectorstore.similarity_search(query, k=max(k_values))
    retrieved_ids = [d.metadata["doc_id"] for d in docs_faiss]

    relevant_ids = get_relevant_doc_ids(split_docs, golden_answer)

    precision_dict = {}
    recall_dict = {}
    map_dict = {}
    ndcg_dict = {}
    for k in k_values:
        precision_dict[f"precision@{k}"] = precision_at_k(retrieved_ids, relevant_ids, k)
        recall_dict[f"recall@{k}"] = recall_at_k(retrieved_ids, relevant_ids, k)
        map_dict[f"map@{k}"] = average_precision(retrieved_ids, relevant_ids, k)
        ndcg_dict[f"ndcg@{k}"] = ndcg_at_k(retrieved_ids, relevant_ids, k)

    context = [d.page_content for d in docs_faiss]
    prediction = ask(query, context)
    rouge = metric_rouge.compute(predictions=[prediction], references=[golden_context])
    bleu = metric_bleu.compute(predictions=[prediction], references=[golden_context])
    longest_match = find_longest_match(prediction, golden_context)
    prediction_golden = ask(query, golden_context)
    rouge_golden = metric_rouge.compute(predictions=[prediction_golden], references=[golden_context])
    bleu_golden = metric_bleu.compute(predictions=[prediction_golden], references=[golden_context])
    longest_match_golden = find_longest_match(prediction_golden, golden_context)

    result = {
        "index": i,
        "rougeL": rouge["rougeL"],
        "rougeL_golden": rouge_golden["rougeL"],
        "bleu": bleu["bleu"],
        "bleu_golden": bleu_golden["bleu"],
        "longest_match": longest_match,
        "longest_match_golden": longest_match_golden,
        "query": query,
        "gold_answer": golden_answer,
        "prediction": prediction,
        "prediction_golden": prediction_golden,
        "retrieved_docs": context,
        "relevant_ids": relevant_ids,
        "retrieved_ids": retrieved_ids,
        **precision_dict,
        **recall_dict,
        **map_dict,
        **ndcg_dict
    }
    return result

### Full benchmark loop

Run the evaluation until the target number of valid samples with answers is reached, skipping samples with empty references.

In [ ]:
num_valid_examples = config["num_valid_examples"]

results = []
i = 0

with tqdm(total=num_valid_examples) as pbar:
    while len(results) < num_valid_examples and i < len(dataset):
        sample = dataset[i]
        
        start_time = time.perf_counter()
        result = process_sample(i, sample)
        elapsed_time = time.perf_counter() - start_time
            
        if result:
            result["elapsed_time"] = elapsed_time
            results.append(result)
            pbar.update(1)
            
        i += 1

print(f"Processed {len(results)} valid samples out of {i} total samples")

### Save and display results

Print average metric scores, save data to CSV and JSON.

In [ ]:
for r in results:
    print(f"\n\n===================================== SAMPLE {r['index']} =====================================")
    print(f"QUERY: {r['query']}\n")
    print(f"PREDICTION: {r['prediction']}\n")
    print(f"PREDICTION (GOLDEN): {r['prediction_golden']}\n")
    print(f"GOLDEN ANSWER: {r['gold_answer']}\n")
    print(f"rougeL: {r['rougeL']}, rougeL_golden: {r['rougeL_golden']}")
    print(f"bleu: {r['bleu']}, blue golden: {r['bleu_golden']}")
    print(f"longest match: {r['longest_match']}, longest match golden: {r['longest_match_golden']}")
    print("retrieved_ids:", r['retrieved_ids'])
    print("relevant_ids:", r['relevant_ids'])
    k_values = config.get("k_values")
    for k in k_values:
        print(f"precision@{k}: {r.get(f'precision@{k}', 'NA')}, recall@{k}: {r.get(f'recall@{k}', 'NA')}, map@{k}: {r.get(f'map@{k}', 'NA')}, ndcg@{k}: {r.get(f'ndcg@{k}', 'NA')}")

In [ ]:
output_folder = config["output_folder"]
run_name = f"run_custom_{datetime.now().strftime('%Y_%m_%d_%H%M%S')}"
out_path = os.path.join(output_folder, run_name)
os.makedirs(out_path, exist_ok=True)

results_df = pd.DataFrame(results)

response = []
for _, row in results_df.iterrows():
    item = {
        "index": row["index"],
        "query": row["query"],
        "gold_answer": row["gold_answer"],
        "prediction": row["prediction"],
        "prediction_golden": row["prediction_golden"],
        "elapsed_time": row["elapsed_time"],
        "retrieved_docs": row["retrieved_docs"],
        "relevant_ids": row["relevant_ids"],
        "retrieved_ids": row["retrieved_ids"]
    }
    for k in config["k_values"]:
        for metric in ["precision", "recall", "map", "ndcg"]:
            key = f"{metric}@{k}"
            if key in row:
                item[key] = row[key]
    response.append(item)

with open(os.path.join(out_path, "responses.json"), "w") as f:
    json.dump(response, f, indent=4)

with open(os.path.join(out_path, "used_config.json"), "w") as f:
    json.dump(config, f, indent=4)

results_df.drop(
    ["prediction_golden", "prediction", "query", "gold_answer", "elapsed_time", "retrieved_docs", "relevant_ids", "retrieved_ids"],
    axis=1, inplace=True
)
file_path = os.path.join(out_path, f"results.csv")
results_df.to_csv(file_path, index=False)

metrics = ["precision", "recall", "map", "ndcg"]
k_values = config["k_values"]

rows = []
for k in k_values:
    row = {"k": k}
    for metric in metrics:
        key = f"{metric}@{k}"
        if key in results_df:
            row[metric] = results_df[key].mean()
        else:
            row[metric] = float('nan')
    rows.append(row)

metrics_by_k_df = pd.DataFrame(rows)
metrics_by_k_df.to_csv(os.path.join(out_path, "metrics_by_k.csv"), index=False)